<a href="https://colab.research.google.com/github/gleangphibul/PDF-Data-Extraction/blob/main/Garn_PDFTextExtraction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# Author: Phuwadol (Garn) Leangphibul

In [1]:
# Install necessary packages (Run Once)

!pip install pikepdf==8.3.0
!pip install ocrmypdf
!pip install PyMuPDF -q

! apt install tesseract-ocr
! apt install libtesseract-dev

!pip install pytesseract
!apt install ghostscript

!pip install tabula-py

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 12.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.5/106.5 kB 4.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 163.0/163.0 kB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 77.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 82.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 48.1 MB/s eta 0:00:00
  Created wheel for img2pdf: filename=img2pdf-0.6.1-py3-none-any.whl size=51001 sha256=66af75d92de9ffcee9c7615575943b996e7d7b76906c84c7fdabdd1fd4ff25b0
  Stored in directory: /root/.cache/pip/wheels/a5/05/56/c05447973db749cd2178b8f95e36f007f0af5f5dce2c6197a5
Successfully built img2pdf
  Attempting uninstall: pikepdf
    Found existing installation: pikepdf 8.3.0
    Uninstalling pikepdf-8.3.0:
      Successfully uninstalled pikepdf-8.3.0
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.

In [3]:
# Import packages
import os
import requests
import ocrmypdf
import subprocess # used to run ocrmypdf
import shutil # used to replace original file with ocr-ed file
import pymupdf # used to extract text from pdf
import fitz # dependency for pymupdf
import tabula # used to extract tables
import pandas as pd # for data frame data structure, used for easy conversion to excel
from google.colab import drive

In [4]:
# Mount to Google Drive (gives code access to files in Google Drive)

drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
# Set up

# Find path to folder with all pdfs
folder_path = '/content/drive/My Drive/PDF_Text_Extraction_Project/PDFs'

# Create an empty list to store texts extracted from pdf
# Each element stores all the text for a pdf
#pdf_texts = []


In [6]:
# Install Java
!apt-get update
!apt-get install -y openjdk-11-jdk-headless

# Set JAVA_HOME environment variable
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"

# Verify installation
!java -version

# Now try tabula again
import tabula

folder_path = '/content/drive/My Drive/PDF_Text_Extraction_Project/PDFs'
files = os.listdir(folder_path)

if files:
    test_file = os.path.join(folder_path, files[0])
    print(f"Processing: {test_file}")

    dfs = tabula.read_pdf(test_file, pages='1')
    print(f"Found {len(dfs)} tables")

Get:1 https://cli.github.com/packages stable InRelease [3,917 B]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:4 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:5 https://cli.github.com/packages stable/main amd64 Packages [344 B]
Hit:6 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:7 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:8 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [2,102 kB]
Get:9 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Hit:10 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:11 https://r2u.stat.illinois.edu/ubuntu jammy/main amd64 Packages [2,822 kB]
Hit:12 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:13 http://security.ubuntu.com/ubuntu jammy-security/univ

Processing: /content/drive/My Drive/PDF_Text_Extraction_Project/PDFs/TestDoc1.pdf
Found 1 tables


In [7]:
# Troubleshooting: Checking if folder path is correct

def path_exists(f_path):
  if os.path.exists(f_path):
    print("Directory now exists!")
    return True
  else:
    print("Failed to create directory")
    return False

In [8]:
path_exists(folder_path)

Directory now exists!


True

In [9]:
# Finds the last page number of a table
# Inputs: folder_p <-- folder path, file_i <-- file index (used for looping), start_p <-- start page number
# Returns an integer: the lasst page number a table extends across
def find_last_page(folder_p, file_i, start_p):
  print("Running find_last_page()")
  #print(folder_p, file_i, start_p)

  # Gets all the files in a given folder
  files = os.listdir(folder_p)

  # Initialize variables
  page_num = start_p
  final_page = False

  # If the folder is not empty
  if files:
    # Get the file
    file = os.path.join(folder_p, files[file_i])
    print(f"Processing: {file}")

    # While not on the final page
    # Increment and keep track of the page number
    while final_page == False:

      # Grabs the current page's table temporarily to scan info and check if it's last page
      temp_dfs = tabula.read_pdf(file, lattice=True, pages=page_num)
      pos1 = temp_dfs[0].iloc[-1,0] # Last row, first column
      pos2 = temp_dfs[0].iloc[-1, 1] # Last row, second column

      # Check if the table has ended by seeing if somewhere in the last row
      # the word 'total' is seen
      if pos1 == 'Total' or pos2 == 'Total':
        final_page = True
        break

      page_num += 1

    print("Final page?", final_page)
    print("Last page:", page_num)

    return page_num


In [10]:
# Extracts table data and stores it as a list of Pandas dataframes
# First call on find_last_page() to see how many pages the table extends across
# Then, extract table data within the page range
# Returns a list of Pandas dataframes (for later merging)
# Inputs: folder_p <-- folder path, file_i <-- file index (used for looping), start_p <-- start page number
def extract_tables(folder_p, file_i, start_p):
  print("Running extract_tables()")

  # Gets all the files in a given folder
  files = os.listdir(folder_p)

  # If folder is not empty
  if files:
    # Get the file and page_range
    file = os.path.join(folder_p, files[file_i])
    last_page_num = find_last_page(folder_p, file_i, start_p)

    page_range = f'{start_p}-{last_page_num}'

    # Extracts the text data on each page the table is in
    dfs = tabula.read_pdf(file, lattice=True, pages=page_range)

    print("Tables collected:", len(dfs))

    return dfs


In [11]:
# Fixes the misprocessing where first row becomes column names
# Grabs the first row data from the column names and make it a mergable row
# Then, return the row
def get_first_row(df):
  print("Running get_first_row()")

  # Create the first row as a list: the name, followed by empty cells
  first_row = [df.columns[0]]
  first_row.extend([""] * (df.shape[1] - 1))

  # Turn it into a mergable row (dataframe type)
  first_row_df = pd.DataFrame([first_row], columns=df.columns)

  return first_row_df





In [12]:
# Fix second row to repeat values for subcells.
# Ex: Recruiting Agency, Recruiting Agency, Recruiting agency, Individual-New, ...
# Then, returns the row
def get_second_row(df):

  print("Running get_second_row()")
  print("Dataframe size:", df.shape)

  # Copy the values over from the second row
  second_row_copy = df.iloc[0]

  # Drop 'nan' values
  cleaned_second_row = second_row_copy.dropna()

  # Create the second row as a list
  second_row_data = cleaned_second_row.tolist()
  print(second_row_data) # Debugging statement

  # List form of the second row
  second_row_list = ["", ""]

  # Get each cell value in the original second row and write it 3 times
  # in the new second row (this one will replace the original second row)
  for value in second_row_data:
    second_row_list.extend([value] * 3)
    #second_row_list.extend([""]) # Temp line of code: ensure column sizes match

  print("Length of second_row_list", len(second_row_list))

  # Turn the list back into a dataframe
  second_row_df = pd.DataFrame([second_row_list], columns=df.columns)

  return second_row_df

In [13]:
# Fixes column misalignment with 'total' in the last row
# Returns a properly reformatted df (last dataframe in the list)
def realign_last_row(df):

  print("Running realign_last_row()")

  print("df shape:", df.shape)

  # Copy data from last row of data frame
  # Drop any weird values (e.g. null values)
  # Transform it into a list (lists are easy to work with)
  # Shift 'total' one column to the right and make necessary adjustments
  # Convert last row back to dataframe
  # Then replace the original last row with the reformatted last row
  last_row_copy = df.iloc[-1]

  cleaned_last_row = last_row_copy.dropna()

  last_row_list = cleaned_last_row.tolist()

  # Reformatting
  last_row_list.insert(0, "")
  last_row_list.append("")  # Add another blank element at the end so that column numbers match

  print(last_row_list)
  print("Length of last row list:", len(last_row_list))

  # Convert back to dataframe
  last_row_df = pd.DataFrame([last_row_list], columns=df.columns)

  # Replace by dropping the last row and replacing with our reformatted last row
  df.drop(df.tail(1).index, inplace=True)

  last_df_final = pd.concat([df, last_row_df], ignore_index=True)

  return last_df_final



In [14]:
# Gets a properly merged and formatted dataframe based on the file and start page
# Inputs: folder_p <-- folder path, file_i <-- file index (used for looping), start_p <-- start page number,
#         one_page <-- bolean whether the table is only on one page
def get_merged_df(folder_p, file_i, start_p, one_page):
  print("Running get_merged_df()")

  if one_page:
    df_list = extract_tables(folder_p, file_i, start_p)

    first_row = get_first_row(df_list[0])
    second_row = get_second_row(df_list[0])
    remaining_first_table = df_list[0].iloc[1:]

    df_merge_list = [first_row, second_row, remaining_first_table]

  else:

    # First get a table (likely spread across multiple pages)
    # Store this as a list where each element is the part of the table on each page
    df_list = extract_tables(folder_p, file_i, start_p)

    # Get the properly formatted first and second rows
    first_row = get_first_row(df_list[0])
    second_row = get_second_row(df_list[0])

    # We don't need the first two rows anymore so just grab the remaining rows
    # from the first table
    remaining_first_table = df_list[0].iloc[1:]

    # Fix the column alignment in the last page with 'total' as well
    # Assume the last row is misaligned
    try:
      last_table = realign_last_row(df_list[-1])
      fixed_last_table = last_table.iloc[2:]
    # The last row was actually properly aligned
    except:
      last_table = df_list[-1]
      fixed_last_table = last_table.iloc[2:]

    # Now we are ready to merge these dataframes together as a singular table
    # Start with the first table which includes: properly formatted first two rows + remaining rows
    df_merge_list = [first_row, second_row, remaining_first_table]

    # Now loop through other pages and append those first
    # Start at the second page (index 1) because we have the first page data already
    # End at the second last page (index length - 1) because we have fixed_last_table
    for i in range(1, len(df_list) - 1):
      df = df_list[i].iloc[2:]
      df_merge_list.append(df)

    # Once all other pages are handled, append table from the last page
    df_merge_list.append(fixed_last_table)

  # Once everything (e.g. pages) is ordered correctly, append the separate dataframes
  final_df = pd.concat(df_merge_list)

  return final_df



In [15]:
# Takes a dataframe data structure and exports it to a csv file
# Stores it in whatever file path specified (currently a google drive folder)
# Inputs: df <-- the dataframe, name <-- what you want to name the file
def export_file_to_csv(df, name):

  print("Running export_file_to_csv()")

  root_store_path = '/content/drive/My Drive/PDF_Text_Extraction_Project/Excel_Tables'
  store_path = os.path.join(root_store_path, f"{name}.csv")

  df.to_csv(store_path)

In [16]:
# Loop function

def main(folder_p):

  # Gets all the files in a given folder
  files = os.listdir(folder_p)

  # If folder is not empty
  if files:
    # i now acts as the file index in the folder
    for i in range(len(files)):
      print("Processing file:", files[i])

      ## For Country-wise Table
      start_page = 1
      one_page = False

      # First check if the table is only on one page
      last_page = find_last_page(folder_p, i, start_page)
      if start_page == last_page:
        one_page = True

      country_df = get_merged_df(folder_p, i, start_page, one_page)
      temp_file_name = files[i].split('.')
      file_name = temp_file_name[0] + "_country"
      export_file_to_csv(country_df, file_name)

      # Reset the one_page variable
      one_page = False
      # Update the start page to be the next page from where we left off
      start_page = last_page + 1

      ## For District-wise Table
      # First check if the table is only on one page
      # last_page = find_last_page(folder_p, i, start_page)
      # if start_page == last_page:
      #   one_page = True

      # district_df = get_merged_df(folder_p, i, start_page, one_page)
      # temp_file_name = files[i].split('.')
      # file_name = temp_file_name[0] + "_district"
      # export_file_to_csv(district_df, file_name)







In [17]:
test = main(folder_path)

Processing file: TestDoc1.pdf
Running find_last_page()
Processing: /content/drive/My Drive/PDF_Text_Extraction_Project/PDFs/TestDoc1.pdf
Final page? True
Last page: 1
Running get_merged_df()
Running extract_tables()
Running find_last_page()
Processing: /content/drive/My Drive/PDF_Text_Extraction_Project/PDFs/TestDoc1.pdf
Final page? True
Last page: 1
Tables collected: 1
Running get_first_row()
Running get_second_row()
Dataframe size: (101, 23)
['Recruiting Agency', 'Individual-New', 'G-to-G', 'Individual-ReEntry', 'Legalization', 'Total with ReEntry', 'Total without ReEntry']
Length of second_row_list 23
Running export_file_to_csv()
Processing file: TestDoc2.pdf
Running find_last_page()
Processing: /content/drive/My Drive/PDF_Text_Extraction_Project/PDFs/TestDoc2.pdf
Final page? True
Last page: 2
Running get_merged_df()
Running extract_tables()
Running find_last_page()
Processing: /content/drive/My Drive/PDF_Text_Extraction_Project/PDFs/TestDoc2.pdf
Final page? True
Last page: 2
Tables

Nov 04, 2025 11:43:00 PM org.apache.pdfbox.pdmodel.font.FileSystemFontProvider <init>
Nov 04, 2025 11:43:00 PM org.apache.pdfbox.pdmodel.font.FileSystemFontProvider <init>
Nov 04, 2025 11:43:00 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
Nov 04, 2025 11:43:01 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>

Nov 04, 2025 11:43:05 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>



Final page? True
Last page: 2
Running get_merged_df()
Running extract_tables()
Running find_last_page()
Processing: /content/drive/My Drive/PDF_Text_Extraction_Project/PDFs/Document_2025052704090.pdf


Nov 04, 2025 11:43:08 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>

Nov 04, 2025 11:43:12 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>



Final page? True
Last page: 2


...

Final page? True
Last page: 2
Running get_merged_df()
Running extract_tables()
Running find_last_page()
Processing: /content/drive/My Drive/PDF_Text_Extraction_Project/PDFs/Document_2025011903430.pdf


Nov 04, 2025 11:43:41 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
Nov 04, 2025 11:43:41 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>

Nov 04, 2025 11:43:44 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
Nov 04, 2025 11:43:44 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>



Final page? True
Last page: 2


Nov 04, 2025 11:43:47 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
Nov 04, 2025 11:43:47 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>



Tables collected: 2
Running get_first_row()
Running get_second_row()
Dataframe size: (79, 20)
['Recruiting Agency', 'Individual-New', 'G-to-G', 'Individual-ReEntry', 'Total with ReEntry', 'Total without ReEntry']
Length of second_row_list 20
Running realign_last_row()
df shape: (26, 20)
['', 'Total', '26618', '2698', '29316', '7742', '2343', '10085', '619', '35', '654', '25174', '2029', '27203', '60153', '7105', '67258', '34979', '5076', '40055', '']
Length of last row list: 21
Running export_file_to_csv()


In [18]:
test_last_page = find_last_page(folder_path, 0, 1)

Running find_last_page()
Processing: /content/drive/My Drive/PDF_Text_Extraction_Project/PDFs/TestDoc1.pdf
Final page? True
Last page: 1


In [19]:
def runocr(file):
    """Run OCR on a PDF file, replace it and rename with OCR_ prefix"""

    # Extract directory and filename
    directory = os.path.dirname(file)
    original_filename = os.path.basename(file)

    # Create new filename with OCR_ prefix
    new_filename = "OCR_" + original_filename
    new_file_path = os.path.join(directory, new_filename)

    print(f"Processing: {original_filename}")
    print(f"Will rename to: {new_filename}")

    # Run OCR to temporary file first
    temp_file = "temp_ocr_output.pdf"
    result = subprocess.run(
        ['ocrmypdf', '--force-ocr', file, temp_file],
        capture_output=True,
        text=True
    )

    if result.returncode == 0:
        # Remove original file
        os.remove(file)
        # Rename temporary file to new name
        shutil.move(temp_file, new_file_path)
        print(f"✓ OCR successful! Renamed to: {new_filename}")

        # Open and return the renamed file
        return_file = fitz.open(new_file_path)
        return return_file
    else:
        print("✗ OCR failed! Return code:", result.returncode)
        if result.stderr:
            print("Error:", result.stderr)
        # Clean up temporary file if it exists
        if os.path.exists(temp_file):
            os.remove(temp_file)
        return None

In [20]:
from sys import ps1
def get_pdf_text(pdf_file):

  num_pages = pdf_file.page_count
  pdf_text = ""
  for i in range(0, num_pages):
    page = pdf_file.load_page(i)
    page_text = page.get_text()

    pdf_text += page_text

  return pdf_text

In [21]:
def run_text_extraction(text_list):
  files = os.listdir(folder_path)

  if files:
    for file in files:
      if "OCR" not in file:
        print(file, "not yet OCR-ed")
        file_path = os.path.join(folder_path, file)
        print(f"Processing: {file_path}")

        my_ocr_file = runocr(file_path)

        if my_ocr_file:
          print(f"Success! Processed PDF has {len(my_ocr_file)} pages")
          pdf_text = get_pdf_text(my_ocr_file)
          text_list.append(pdf_text)
        else:
          print("OCR processing failed")
      else:
        print(file, "OCR-ed already")
        file_path = os.path.join(folder_path, file)
        my_file = fitz.open(file_path)
        pdf_text = get_pdf_text(my_file)
        text_list.append(pdf_text)
  else:
    print("No files found")




In [22]:
def download_file(url):
    local_filename = url.split('/')[-1]

    with requests.get(url) as r:
        assert r.status_code == 200, f'error, status code is {r.status_code}'
        with open(local_filename, 'wb') as f:
            f.write(r.content)

    return local_filename
